# IU source table to Cassandra

1. 주피터 노트북으로 사용할 때
    - Jupyter Notebook 생성시 init script에 PYSPARK_SUBMIT_ARGS 추가 필요
        - --packages com.datastax.spark:spark-cassandra-connector_2.11:2.5.1
        - --conf spark.sql.extensions=com.datastax.spark.connector.CassandraSparkExtensions
        - --conf spark.cassandra.connection.host=10.128.168.146  *(Cassandra IP 변경시 수정 필요)*
    - Jupyter Notebook 생성시 IU 키탭 등록 (개발 기준)
        - 키탭 계정명 입력: linewallet_ml_dev
        - 키탭 파일 업로드: linewallet_ml_dev 
2. Airflow 의 MLUPapermill operator 로 사용할 때
    - PYSPARK_SUBMIT_ARGS 추가: https://wiki.linecorp.com/display/MLU/%5BMLU+spec%5D+Default+argument+-+jupyter_cfg
    - IU 키탭 등록: https://wiki.linecorp.com/display/MLU/%5BMLU+spec%5D+Default+argument+-+cluster_cfg  

## 주의
**Paragraph 가 실행되지 않고 Kernel Restarting 이 발생할 경우, Shut Down Kernel 후 Restart Kernel...**

## 0. Cassandra Keyspace 및 Table 생성

### cqlsh 실행 예시

*TODO: cassandra password 설정 필요*
```console
$ pip install cqlsh
$ cqlsh 10.128.168.146
```

### keyspace 및 table 생성 cql 예시 (cqlsh 로 실행 필요)

*TODO: class 및 replication_factor 수정 필요*
```console
cqlsh> CREATE KEYSPACE IF NOT EXISTS featurestore WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': '1'};
```

```console
cqlsh> CREATE TABLE IF NOT EXISTS featurestore.wrs_user_demo 
( mid text
, sex text
, age_range text
, age_group text
, log_date text
, region text
, PRIMARY KEY ((log_date, region))
);
```

## 1. IU source table 데이터 로드

In [ ]:
date = '20220707'
dummy_param = 'param not injected'

In [ ]:
print("---------------------------------------")
print(dummy_param)
print("---------------------------------------")

In [3]:
sql_load_source = """
select mid, sex, age_range, age_group, log_date, region
from linewallet_ml_dev.wrs_user_demo
where 1=1
and log_date = '{date}'
and region = 'JP'
limit 100
""".format(date=date)

In [4]:
df_source = spark.sql(sql_load_source)

In [5]:
df_source.head()

Row(mid='u8666febbec3dd517aea4b907470242d4', sex='f', age_range='50-', age_group='50', log_date='20220707', region='JP')

## 2. Cassandra 테이블에 IU source table 데이터 적재

In [6]:
df_source\
    .write\
    .format("org.apache.spark.sql.cassandra")\
    .mode("append")\
    .options(table="wrs_user_demo", keyspace="featurestore")\
    .save()

## 3. Cassandra 테이블 확인

In [6]:
df_cassandra = spark\
                .read\
                .format("org.apache.spark.sql.cassandra")\
                .options(table="wrs_user_demo", keyspace="featurestore")\
                .load()

In [7]:
df_cassandra.head()

Row(log_date='20220707', region='JP', age_group='50', age_range='50-', mid='ue7a17910c67428b5d57f5e587b554390', sex='m')